<a href="https://colab.research.google.com/github/toan-ly/Image-Retrieval/blob/feature%2Ftraditional_retrieval/Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm

!apt-get update
!apt-get install -y wget
!pip install selenium
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup # for parsing HTML content
from urllib.parse import urljoin, urlparse # for handling URLs
import urllib.request # for making HTTP requests
import time # for handling time-related operations
import os # for interacting with the operating system (related to dir, folder, file)
from tqdm import tqdm
import concurrent.futures # for multi-threading
import json # for writing to a text file
from PIL import Image

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,553 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,423 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,447 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,133 kB]
Get:14 

In [2]:
url = 'https://www.flickr.com/search/?text='
search_term = 'cat'

In [3]:
class UrlScraper:
    def __init__(self, url_template, max_images=50, max_workers=4):
        self.url_template = url_template # link crawl
        self.max_images = max_images # max images
        self.max_workers = max_workers # max threads
        self.setup_environment()

    # Set up environment for selenium
    def setup_environment(self):
        os.environ['PATH'] += ':/usr/lib/chromium-browser/'
        os.environ['PATH'] += ':/usr/lib/chromium-browser/chromedriver/'

    def get_url_images(self, term):
        """
        Crawl the urls of images by term

        Parameters:
        term (str): The name of animal, plant, scenery, furniture

        Returns:
        urls (list): List of urls of images
        """

        # Initialize Chrome driver
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=options)

        url = self.url_template.format(search_term=term)
        driver.get(url)

        # Start crawl urls of image like brute force - the same mechanism with this but add some features
        urls = []
        more_content_available = True

        pbar = tqdm(total=self.max_images, desc=f'Fetching images for {term}')

        while len(urls) < self.max_images and more_content_available:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            img_tags = soup.find_all('img')

            for img in img_tags:
                if len(urls) >= self.max_images:
                    break
                if 'src' in img.attrs:
                    href = img.attrs['src']
                    img_path = urljoin(url, href)
                    img_path = img_path.replace('_m.jpg', '_b.jpg').replace('_n.jpg', '_b.jpg').replace('_w.jpg', '_b.jpg')
                    if img_path == "https://combo.staticflickr.com/ap/build/images/getty/IStock_corporate_logo.svg":
                        continue
                    urls.append(img_path)
                    pbar.update(1)
            try:
                load_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@id="yui_3_16_0_1_1721642285931_28620"]'))
                )
                load_more_button.click()
                time.sleep(2)
            except:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)

                new_soup = BeautifulSoup(driver.page_source, 'html.parser')
                new_img_tags = new_soup.find_all('img', loading='lazy')
                if len(new_img_tags) == len(img_tags):
                    more_content_available = False
                img_tags = new_img_tags

        pbar.close()
        driver.quit()
        return urls

    def scrape_urls(self, categories):
        """
        Call get_url_images method to get all urls of any object in categories\

        Parameter:
        categories (dictionary): the dict of all object we need to collect image with format categories{"name_object": [value1, value2, ...]}

        Returns:
        all_urls (dictionary): Dictionary of urls of images
        """
        all_urls = {category: {} for category in categories}

        # Handle multi-threading for efficent installation
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            future_to_term = {executor.submit(self.get_url_images, term): (category, term)
                              for category, terms in categories.items() for term in terms}

            for future in tqdm(concurrent.futures.as_completed(future_to_term), total=len(future_to_term), desc="Overall Progress"):
                category, term = future_to_term[future]
                try:
                    urls = future.result()
                    all_urls[category][term] = urls
                    print(f"\nNumber of images retrieved for {term}: {len(urls)}")
                except Exception as exc:
                    print(f"\n{term} generated an exception: {exc}")
        return all_urls

    def save_to_file(self, data, filename):
        """
        Save the data to a JSON file.

        Parameters:
        data (dict): The data to be saved.
        filename (str): The name of the JSON file.

        Returns:
        None
        """
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"Data saved to {filename}")


In [4]:
categories = {
    "animal": ["Monkey", "Elephant", "cows", "Cat", "Dog", "bear", "fox", "Civet", "Pangolins", "Rabbit", "Bats", "Whale", "Cock", "Owl", "flamingo", "Lizard", "Turtle", "Snake", "Frog", "Fish", "shrimp", "Crab", "Snail", "Coral", "Jellyfish", "Butterfly", "Flies", "Mosquito", "Ants", "Cockroaches", "Spider", "scorpion", "tiger", "bird", "horse", "pig", "Alligator", "Alpaca", "Anteater", "donkey", "Bee", "Buffalo", "Camel", "Caterpillar", "Cheetah", "Chicken", "Dragonfly", "Duck", "panda", "Giraffe"],
    "plant": ["Bamboo", "Apple", "Apricot", "Banana", "Bean", "Wildflower", "Flower", "Mushroom", "Weed", "Fern", "Reed", "Shrub", "Moss", "Grass", "Palmtree", "Corn", "Tulip", "Rose", "Clove", "Dogwood", "Durian", "Ferns", "Fig", "Flax", "Frangipani", "Lantana", "Hibiscus", "Bougainvillea", "Pea", "OrchidTree", "RangoonCreeper", "Jackfruit", "Cottonplant", "Corneliantree", "Coffeeplant", "Coconut", "wheat", "watermelon", "radish", "carrot"],
    "furniture": ["bed", "cabinet", "chair", "chests", "clock", "desks", "table", "Piano", "Bookcase", "Umbrella", "Clothes", "cart", "sofa", "ball", "spoon", "Bowl", "fridge", "pan", "book"],
    "scenery": ["Cliff", "Bay", "Coast", "Mountains", "Forests", "Waterbodies", "Lake", "desert", "farmland", "river", "hedges", "plain", "sky", "cave", "cloud", "flowergarden", "glacier", "grassland", "horizon", "lighthouse", "plateau", "savannah", "valley", "volcano", "waterfall"]
}

In [5]:
urltopic = {"flickr": "https://www.flickr.com/search/?text={search_term}"}
scraper = UrlScraper(url_template=urltopic["flickr"], max_images=20, max_workers=5)
image_urls = scraper.scrape_urls(categories)

Fetching images for Monkey:   0%|          | 0/20 [00:00<?, ?it/s]

Fetching images for Monkey:   5%|▌         | 1/20 [00:02<00:52,  2.75s/it]


Fetching images for Elephant:   0%|          | 0/20 [00:00<?, ?it/s]



Fetching images for Cat:   0%|          | 0/20 [00:00<?, ?it/s]

Fetching images for Dog:   5%|▌         | 1/20 [00:02<00:38,  2.03s/it]


Fetching images for Elephant:   5%|▌         | 1/20 [00:01<00:28,  1.52s/it]



Fetching images for Cat:   5%|▌         | 1/20 [00:01<00:25,  1.33s/it]




Fetching images for cows:   0%|          | 0/20 [00:00<?, ?it/s]




Overall Progress:   1%|          | 1/134 [01:05<2:25:47, 65.77s/it]


Number of images retrieved for Monkey: 20


Fetching images for Elephant: 100%|██████████| 20/20 [00:17<00:00,  1.13it/s]


Number of images retrieved for Dog: 20

Number of images retrieved for Cat: 20



Overall Progress:   3%|▎         | 4/134 [01:10<24:14, 11.19s/it]  

...


Number of images retrieved for volcano: 20


Overall Progress:  99%|█████████▉| 133/134 [14:51<00:04,  4.43s/it]


Number of images retrieved for valley: 20


Overall Progress: 100%|██████████| 134/134 [14:53<00:00,  6.67s/it]


Number of images retrieved for waterfall: 20


In [6]:
scraper.save_to_file(image_urls, 'image_urls.json')

Data saved to image_urls.json


In [7]:
class ImageDownloader:
    def __init__(self, json_file, download_dir='Dataset', max_workers=4, delay=1):
        self.json_file = json_file  # file containing URLs of images in JSON format
        self.download_dir = download_dir  # Folder name for storing images
        self.max_workers = max_workers  # Number of threads
        self.delay = delay  # Polite delay: when we send request too much to the server for downloading images without polite delay, it will crash or prevent your IP
        self.filename = set()  # To store filename directories
        self.setup_directory()  # Set up the folder structure

    def setup_directory(self):
        if not os.path.exists(self.download_dir):
            os.makedirs(self.download_dir)

    def read_json(self):
        """
        Read the JSON file and return the data.

        Returns:
        data (dict): The data read from the JSON file.
        """
        with open(self.json_file, 'r') as file:
            data = json.load(file)
        return data

    def is_valid_url(self, url):
        """
        Check if the URL is valid.

        Parameters:
        url (str): The URL to be checked.

        Returns:
        bool: True if the URL is valid, False otherwise.
        """
        try:
            with urllib.request.urlopen(url) as response:
                if response.status == 200 and 'image' in response.info().get_content_type():
                    return True
        except Exception:
            return False

    def download_image(self, url, category, term, pbar):
        """
        Download the image from the given URL.

        Parameters:
        url (str): The URL of the image to be downloaded.
        category (str): The category of the image.
        term (str): The term or keyword associated with the image.
        pbar (tqdm): The progress bar object.

        Returns:
        str: A message indicating the status of the download.
        """
        if not self.is_valid_url(url):
            pbar.update(1)
            return f"Invalid URL: {url}"

        category_dir = os.path.join(self.download_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

        term_dir = os.path.join(category_dir, term)
        if not os.path.exists(term_dir):
            os.makedirs(term_dir)

        filename = os.path.join(term_dir, os.path.basename(urlparse(url).path))

        self.filename.add(filename)  # Record the filename directory

        try:
            urllib.request.urlretrieve(url, filename)
            pbar.update(1)
            return f"Downloaded: {url}"
        except Exception as e:
            pbar.update(1)
            return f"Failed to download {url}: {str(e)}"

    def download_images(self):
        """
        Download images from the JSON file.

        Returns:
        None
        """
        data = self.read_json()
        download_tasks = []

        total_images = sum(len(urls) for terms in data.values() for urls in terms.values())
        with tqdm(total=total_images, desc="Downloading images") as pbar:
            with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                for category, terms in data.items():
                    for term, urls in terms.items():
                        for url in urls:
                            download_tasks.append(executor.submit(self.download_image, url, category, term, pbar))
                            time.sleep(self.delay)  # Polite delay

                for future in concurrent.futures.as_completed(download_tasks):
                    print(future.result())

        self.export_filename()

    def export_filename(self):
        """
        Export the filename directories to a text file.

        Returns:
        None
        """
        with open('filename.txt', 'w') as file:
            for filename in sorted(self.filename):
                file.write(f"{filename}\n")


In [ ]:
downloader = ImageDownloader(json_file='image_urls.json', download_dir='Dataset', max_workers=4, delay=1)
downloader.download_images()

In [9]:
downloader.export_filename()

In [10]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [11]:
!ls

Dataset  drive	filename.txt  image_urls.json  sample_data


In [12]:
!zip -r /content/drive/MyDrive/Dataset.zip Dataset

  adding: Dataset/ (stored 0%)
  adding: Dataset/animal/ (stored 0%)
  adding: Dataset/animal/shrimp/ (stored 0%)
  adding: Dataset/animal/shrimp/4898039985_e3eaca1c87_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/2144708533_8f37851185_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/17061896855_554c2b2b8a_b.jpg (deflated 1%)
  adding: Dataset/animal/shrimp/2344784902_8c2ddbed93_b.jpg (deflated 1%)
  adding: Dataset/animal/shrimp/233169073_da697f9b1b_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/53624724282_71047360df_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/39860909861_ac4dc6142b_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/3553852023_779b809b27_b.jpg (deflated 2%)
  adding: Dataset/animal/shrimp/52754170014_d0570ded3f_b.jpg (deflated 1%)
  adding: Dataset/animal/shrimp/50807936448_b99cb05879_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/50861234561_6cc092f2e5_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/15364085118_8a99f61fc8_b.jpg (defla

In [13]:
def check_and_preprocess_images(image_dir):
    """
    Check and preprocess images in the specified directory.

    Parameters:
    image_dir (str): The directory containing the images to be checked and preprocessed.

    Returns:
    None
    """
    for root, _, files in os.walk(image_dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    # Check if image is smaller than 50x50 pixels
                    if img.size[0] < 50 or img.size[1] < 50:
                        os.remove(file_path)
                        print(f"Deleted {file_path}: Image too small ({img.size[0]}x{img.size[1]})")
                        continue

                    # Convert non-RGB images to RGB
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                        img.save(file_path)
                        print(f"Converted {file_path} to RGB")

            except Exception as e:
                # If file is not an image, delete it
                os.remove(file_path)
                print(f"Deleted {file_path}: Not an image or corrupted file ({str(e)})")

In [14]:
check_and_preprocess_images('Dataset')

Converted Dataset/animal/Elephant/4839032364_8c521066b2_b.jpg to RGB
Converted Dataset/animal/Elephant/4839031518_3765f997a4_b.jpg to RGB


In [15]:
!zip -r /content/drive/MyDrive/Clean_Dataset.zip Dataset

  adding: Dataset/ (stored 0%)
  adding: Dataset/animal/ (stored 0%)
  adding: Dataset/animal/shrimp/ (stored 0%)
  adding: Dataset/animal/shrimp/4898039985_e3eaca1c87_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/2144708533_8f37851185_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/17061896855_554c2b2b8a_b.jpg (deflated 1%)
  adding: Dataset/animal/shrimp/2344784902_8c2ddbed93_b.jpg (deflated 1%)
  adding: Dataset/animal/shrimp/233169073_da697f9b1b_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/53624724282_71047360df_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/39860909861_ac4dc6142b_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/3553852023_779b809b27_b.jpg (deflated 2%)
  adding: Dataset/animal/shrimp/52754170014_d0570ded3f_b.jpg (deflated 1%)
  adding: Dataset/animal/shrimp/50807936448_b99cb05879_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/50861234561_6cc092f2e5_b.jpg (deflated 0%)
  adding: Dataset/animal/shrimp/15364085118_8a99f61fc8_b.jpg (defla

In [16]:
!cp filename.txt /content/drive/MyDrive/filename.txt